<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Advanced%20TensorFlow/Extending%20Keras/Week%204%20-%20Models%20and%20Callbacks/ExerciseAnswer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise - Create a VGG network 

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

## Create named-variables dynamically

In [ ]:
# Use the vars keyword to create a local variable named 'class'
vars()['class'] = 'foobar' 

In [ ]:
# Here is how you access the variable
vars()['class']

'foobar'

## Create a generic VGG block
This block subclasses tf.keras.Model and uses some layers like Conv2D, MaxPooling2D

In [ ]:
class Block(tf.keras.Model):
  def __init__(self, filters, kernel_size, repetitions, pool_size=2):
    super(Block, self).__init__()
    self.filters = filters
    self.kernel_size = kernel_size
    self.repetitions = repetitions
    self.max_pool = layers.MaxPool2D((pool_size, pool_size), strides=(2, 2))

    for i in range(self.repetitions):
      vars(self)['block_{}'.format(i)] = layers.Conv2D(filters, kernel_size, activation='relu', padding='same')
  
  def call(self, inputs):
    for i in range(self.repetitions):
      layer = vars(self)['block_{}'.format(i)]
      if i == 0:
        x = layer(inputs)
      else:
        x = layer(x)
    return self.max_pool(x)

# Create the Custom VGG network
This model stack a series of VGG blocks and a classification head is attached in the end.  

In [ ]:
class MyVGG(tf.keras.Model):
  def __init__(self, num_classes):
    super(MyVGG, self).__init__()

    # Creating blocks of VGG with the following 
    # (filters, kernel_size, repetitions) configurations
    self.block_a = Block(64, 3, 2)
    self.block_b = Block(128, 3, 2)
    self.block_c = Block(256, 3, 3)
    self.block_d = Block(512, 3, 3)
    self.block_e = Block(512, 3, 3)

    # Classification head
    self.flatten = layers.Flatten(name='flatten')
    self.fc = layers.Dense(256, activation='relu', name='fc1')
    self.classifier = layers.Dense(num_classes, activation='softmax')

  def call(self, inputs):
    x = self.block_a(inputs)
    x = self.block_b(x)
    x = self.block_c(x)
    x = self.block_d(x)
    x = self.block_e(x)
    x = self.flatten(x)
    x = self.fc(x)
    return self.classifier(x)

In [ ]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN)

# Initialize VGG with the number of classes 
vgg = MyVGG(num_classes=2)

# Compile with losses and metrics
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define preprocessing function
def preprocess(features):
  # Resize and normalize
  image = tf.image.resize(features['image'], (224, 224))
  return tf.cast(image, tf.float32) / 255., features['label']

# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)

# Train the custom VGG model
vgg.fit(dataset, epochs=10)